In [1]:
from qiskit import *
from math import pi
import numpy as np
from qiskit.aqua.operators import CircuitStateFn
from qiskit.aqua.operators.gradients import QFI
from qiskit.circuit import QuantumCircuit, ParameterVector, Parameter
from sklearn import datasets

In [2]:
iris = datasets.load_iris()
X = iris.data[:, :][0:100]
y = iris.target[0:100]

In [3]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

In [4]:
def add_layerRZ(qf, data):
    for i in range(numQubits):
        qf.rz(data[i], i)
    qf.barrier()

def add_layerRZZ(qf, data):
    for i in range(0, numQubits-1):
        for j in range(i+1, numQubits):
            qf.cx(i, j)
            qf.rz((pi-data[i])*(pi-data[j]), j)
            qf.cx(i, j)
        qf.barrier()

def make_feature_map(qf, data):
    for i in range(numQubits):
        qf.h(i)
    qf.barrier()
    add_layerRZ(qf, data)
    add_layerRZZ(qf, data)

In [5]:
def add_cnots(qf):
    for i in range(0, numQubits-1):
        for j in range(i+1, numQubits):
            qf.cx(i, j)
    qf.barrier()

def add_layerRY(qf):
    for qubit in range(numQubits):
        qf.ry(next(theta_iter), qubit)
    qf.barrier()

def make_variationalCircuit(qf):
    add_layerRY(qf)
    add_cnots(qf)
    add_layerRY(qf)

In [6]:
def make_final_circuit(qf, data):
    make_feature_map(qf, data)
    make_variationalCircuit(qf)

In [7]:
def get_parameters():
    return np.random.uniform(-1, 1, numQubits*numParametrisedLayers)

In [8]:
def measure_and_output(qf, parameter_values):
    for j in range(numQubits):
        qf.measure(j, j)
    bitString = execute(qf,backend=BasicAer.get_backend('qasm_simulator'), parameter_binds=parameter_values, shots=1, memory = True).result().get_counts()
    bits = [int(bit) for bit in list(bitString.keys())[0]]
    xor_result = 0
    for i in range(len(bits)):
        xor_result = xor_result^bits[i]
    return xor_result

In [9]:
y_result = [0 for j in range(len(y))]

In [10]:
numQubits = len(X[0])
numParametrisedLayers = 2

In [11]:
for index in range(len(X)):
    
    data = normalize(X[index])
    
    parameters = [Parameter('x{}'.format(i)) for i in range(numQubits*numParametrisedLayers)]
    theta_iter = iter(parameters)
    
    np.random.seed(0)
    values_dict = get_parameters()
    parameter_values = [{x:  values_dict[i] for i in range(len(values_dict)) for x in parameters}]
    
    qf = QuantumCircuit(QuantumRegister(numQubits), ClassicalRegister(numQubits))
    make_final_circuit(qf, data)
    
    y_result[index] = measure_and_output(qf, parameter_values)
    
    print("data : {}, y : {}, y_result : {}".format(data, y[index], y_result[index]))

data : [0.80377277 0.55160877 0.22064351 0.0315205 ], y : 0, y_result : 0
data : [0.82813287 0.50702013 0.23660939 0.03380134], y : 0, y_result : 0
data : [0.80533308 0.54831188 0.2227517  0.03426949], y : 0, y_result : 0
data : [0.80003025 0.53915082 0.26087943 0.03478392], y : 0, y_result : 0
data : [0.790965  0.5694948 0.2214702 0.0316386], y : 0, y_result : 0
data : [0.78417499 0.5663486  0.2468699  0.05808704], y : 0, y_result : 0
data : [0.78010936 0.57660257 0.23742459 0.0508767 ], y : 0, y_result : 0
data : [0.80218492 0.54548574 0.24065548 0.0320874 ], y : 0, y_result : 0
data : [0.80642366 0.5315065  0.25658935 0.03665562], y : 0, y_result : 0
data : [0.81803119 0.51752994 0.25041771 0.01669451], y : 0, y_result : 0
data : [0.80373519 0.55070744 0.22325977 0.02976797], y : 0, y_result : 0
data : [0.786991   0.55745196 0.26233033 0.03279129], y : 0, y_result : 0
data : [0.82307218 0.51442011 0.24006272 0.01714734], y : 0, y_result : 1
data : [0.8025126  0.55989251 0.20529392 0